# Preparations

In [ ]:
import numpy as np
pi = np.pi

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from util import plotstyle
PLOTSTYLE = 'screen_dark'
#PLOTSTYLE = 'print'
plotstyle.load(PLOTSTYLE)


In [ ]:
COLORS = plt.rcParams['axes.prop_cycle'].by_key()['color']
COLORS

In [ ]:
from util.schematics import *

# Back Focal Plane Imaging

In [ ]:
f = 2.5 # mm
n = 1.51
NA = 1.2

alpha = 60*pi/180 # radians

def I(theta):
    t = theta-alpha
    return 0.1 + (2+np.cos(t))/3 * (1+3*np.square(np.cos(t)))/4 * np.square(np.cos(4*t))


In [ ]:
r = f*np.tan(np.arcsin(NA/n))
R = 1.2*r
h_lens = 0.15*R

NA_DFC = 1.4
r_dfc = f*np.tan(np.arcsin(NA_DFC/n))

#def lens(x):
#    return np.square(x/R)*(-h_lens)+h_lens

#def BFP(x):
#    return x*0-f
    
X = np.linspace( -R, R, 400 )
THETA = np.linspace( -pi, pi, 400 )


In [ ]:
ro = 0.2

def obj_x(theta):
    t = theta-alpha
    return np.sin(theta)*(ro)

def obj_y(theta):
    t = theta-alpha
    return -np.cos(theta)*(ro)+f

def mieplot_x(theta):
    t = theta-alpha
    return np.sin(theta)*(ro+I(theta))

def mieplot_y(theta):
    t = theta-alpha
    return -np.cos(theta)*(ro+I(theta))+f
    
def slice_y( x ):
    return f-np.abs(x)*f/r
    

In [ ]:


fig = plt.figure(figsize=(16.5,23.6), dpi=100)

axs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(axs[0, 0],aspect='equal')

#ax.scatter( [0], [f], color=plotstyle.monochrome_fg() ) # focal point

# Draw the Lens
#ax.plot( X, lens(X), color=plotstyle.monochrome_fg() )
#ax.plot( X, -lens(X), color=plotstyle.monochrome_fg() )

# Normal Optical Path
DFC = Lens( 0.15, 0.05 ).translate(0,2*f).scale( 1.2*r_dfc )
OL = Lens( 0.15, 0.15 ).scale(R)
#SP = Plane().translate(1+ro,f)
BFP = Plane().scale(R).translate(0,-f)
TL = Lens( 0.15, 0.15 ).scale(R).translate(0,-7*f)
EMCCD1 = Sensor().scale(R/2).translate(0,-11*f)

# Sideways Path
M1 = Mirror().scale(R/np.sqrt(2)).translate(0,-4*f).rotate(-pi/4)
IL1 = Lens( 0.15, 0.15 ).scale(R/2).translate(3*f,-4*f).rotate(-pi/2)
IL2 = Lens( 0.15, 0.15 ).scale(R/2).translate(5*f,-4*f).rotate(-pi/2)
M2 = Mirror().scale(R/np.sqrt(2)).translate(7*f,-4*f).rotate(3*pi/4)
PLATFORM = Box().translate(6.25*f,-4*f).scale(2*f, R).set_attrib(c=COLORS[4], ls=':')

# Second Downwards Path
SLIT = Slit().scale(R/2, 0.1).translate(7*f, -6*f)
OL3 = Lens( 0.15, 0.15 ).scale(R/2).translate(7*f, -7*f)
TL3 = Lens( 0.15, 0.15 ).scale(R/2).translate(7*f, -8*f)
GRATING = Mirror().scale(R/2).translate(7*f, -10*f)
EMCCD2 = Sensor().scale(R/2).translate(7*f,-11*f)



PLATFORM.draw(ax)
PLATFORM.label(ax, "Linear Translation Stage", 'top')
ax.arrow( np.max(PLATFORM.X()), 0.5*(np.min(PLATFORM.Y())+np.max(PLATFORM.Y())), 1, 0, width=0.1, length_includes_head=True, color=PLATFORM.preferred_colour )
ax.arrow( np.max(PLATFORM.X()), 0.5*(np.min(PLATFORM.Y())+np.max(PLATFORM.Y())), -1, 0, width=0.1, length_includes_head=True, color=PLATFORM.preferred_colour )

EMCCD1.draw(ax)
EMCCD1.label(ax, "EMCCD\nAndor iXon ...")
EMCCD1.label(ax, "$w_\mathrm{Sensor} = 8\,\mathrm{mm}$\n$1\,\mathrm{px}\hat{=}8\,$µm", 'right')
EMCCD2.draw(ax)
EMCCD2.label(ax, "sCMOS\npco.edge 4.2")
EMCCD2.label(ax, "$w_\mathrm{Sensor} \geq 13.3\,\mathrm{mm}$\n$1\,\mathrm{px}\hat{=}6.5\,$µm", 'right')
GRATING.draw(ax)
GRATING.label(ax, "Diffraction Grating")
# Draw the Lenses
#   Objective
OL.draw(ax)
OL.label(ax, "[OL] Objective Lens")
OL.label(ax, "$f_\mathrm{O}=2.5\,\mathrm{mm},~\mathrm{NA}=1.2$", 'right' )
#   dark field Condenser
DFC.draw(ax)
DFC.label(ax, "Dark Field Condenser\n$\mathrm{NA}=1.2...1.4$", 'right')
#   Intermediate 1
IL1.draw(ax)
IL1.label(ax, "[IL1]\nIntermediate Lens", 'top' )
IL1.label(ax, "$f_\mathrm{I1}=400\,\mathrm{mm}$", 'bottom' )
IL1.label(ax, "\nNot available with\n\"AB\" anti-reflection coating", 'bottom', c=COLORS[1] )
#   Intermediate 2
IL2.draw(ax)
IL2.label(ax, "[IL2]\nAC254-200-AB-ML\nIntermediate Lens", 'top' )
IL2.label(ax, "$f_\mathrm{I2}=200\,\mathrm{mm}$", 'bottom' )
#   Obj 3
OL3.draw(ax)
OL3.label(ax, "[OL3]\nAC254-200-AB-ML", 'right' )
OL3.label(ax, "$f_\mathrm{I2}=200\,\mathrm{mm}$", 'left' )
#   Tub 3
TL3.draw(ax)
TL3.label(ax, "[TL3]\nAC254-200-AB-ML", 'right' )
TL3.label(ax, "$f_\mathrm{I2}=200\,\mathrm{mm}$", 'left' )
#   Tube
TL.draw(ax)
TL.label(ax, "Tube Lens")
TL.label(ax, "AC254-250-A1-ML\n$f_\mathrm{T}=250\,\mathrm{mm}$", 'right')

# Draw the BFP
BFP.set_attrib(ls="--").draw(ax)
BFP.label(ax, "[BFP] Back Focal Plane")
BFP.label(ax, "$r_\mathrm{BFP}=f\cdot\mathrm{sin}θ$ or \n$r_\mathrm{BFP}=f\cdot\mathrm{tan}θ$ ?", 'right', c=COLORS[1])
BFP.label(ax, "$w_\mathrm{Img,BFP}=4.48\,\mathrm{mm}$ or \n$w_\mathrm{Img,BFP}=10.0\,\mathrm{mm}$", 'right', c=COLORS[0], m=3.0)

# Draw the Mirrors 
M1.draw(ax)
M1.label(ax, "[M1] Flippable Mirror")
M2.draw(ax)
M2.label(ax, "[M2] Articulated Mirror", 'top')

# Draw the Slit
SLIT.draw(ax)
SLIT.label(ax, "Slit", 'right')
SLIT.label(ax, "$w_\mathrm{max}=6\,\mathrm{mm}$\n$h_\mathrm{Aperture}=12\,\mathrm{mm}$")



# Draw the optical axis
ax.plot( [0,0], [1.5*f, -1.2*f], ls="--", color=plotstyle.monochrome_fg(), alpha=plotstyle.err_alpha(), lw=1 ) 

# Draw the Scatterer
SCATTERER = Polygon(100).scale(ro).translate(0,f)
SCATTERER.draw(ax)
ax.text( -3*ro, f, "Scattering Sample", va='center', ha='right' )
#SCATTERER.label(ax, "Scatterer")

# Draw the Incident Light
ax.arrow( -2*np.sin(alpha), f+2*np.cos(alpha), (2-ro)*np.sin(alpha), (-2+ro)*np.cos(alpha), width=0.05, length_includes_head=True, color=COLORS[3] )
ax.text( -2*np.sin(alpha), f+2*np.cos(alpha), "Incident Light ", va='center', ha='right', rotation=-90+180/pi*alpha, rotation_mode='anchor', color=COLORS[3] )

# Draw the Mie Plot around the scatterer
ax.plot( mieplot_x(THETA), mieplot_y(THETA), color=COLORS[0], lw=1 ) 
_THETA = np.linspace( -np.arcsin(NA/n), np.arcsin(NA/n), 400 )
polygon_x = np.append( mieplot_x(_THETA), np.append( obj_x(-_THETA), mieplot_x(_THETA)[0] ))
polygon_y = np.append( mieplot_y(_THETA), np.append( obj_y(-_THETA), mieplot_y(_THETA)[0] ))
#polygon_y = np.append( mieplot_y(_THETA), [f, mieplot_y(_THETA)[0]] )
ax.fill( polygon_x, polygon_y, alpha=plotstyle.err_alpha() )
ax.text( 6*ro, 0.8*f, "Scattered Light\nIntensity Distribution", va='center', ha='left', color=COLORS[0] )

# Draw the Cone of light that passes through the objective lens
OL.light_cone( f, np.arcsin(NA/n), 0 ).set_attrib(color=COLORS[2]).draw(ax)
#PATH_11 = Curve().scale(0, -7*f).translate(-r, 0).set_attrib(c=COLORS[2],ls=':')
#PATH_11.draw(ax)
#PATH_11.translate(2*r, 0.0).draw(ax)
#_X = np.linspace( -r, r, 400 )
#light_cone_x = np.append( -r, np.append( _X, r ) )
#light_cone_y = np.append( -f, np.append( slice_y(_X), -f ) )
#ax.plot( light_cone_x, light_cone_y, ls=":", color=plotstyle.monochrome_fg() )

#ax.plot( LC1.X(), LC1.Y(),  )

# Draw the Cone of incident light
DFC.light_cone( f, np.arcsin(1.4/n), 1 ).set_attrib(ls=":", color=COLORS[3]).draw(ax)
DFC.light_cone( f, np.arcsin(1.2/n), 1 ).set_attrib(ls=":", color=COLORS[3]).draw(ax)



# Draw the Intensity Distribution in the BFP
_X = f*np.tan(_THETA)
ax.fill_between( _X, BFP(_X), I(_THETA)-f, color=COLORS[0], alpha=plotstyle.err_alpha() )
_thet_lim = np.arctan( R/f )
_THETA = np.linspace( -_thet_lim, _thet_lim, 400 )
_X = f*np.tan(_THETA)
ax.plot( _X, I(_THETA)-f, color=COLORS[0] )


a = 17.5*pi/180
#a = 60*pi/180
ax0 = obj_x(a)
ay0 = obj_y(a)
ax1 = mieplot_x(a)
ay1 = mieplot_y(a)
ax.plot( [ax0, ax1], [ay0, ay1], color=COLORS[1], lw=1, ls='-' )
ax2 = f*np.tan(a)
ay2 = 0
ax3 = f*np.tan(a)
ay3 = -f+I(a)
ax.plot( [ax1, ax2, ax3], [ay1, ay2, ay3], color=COLORS[1], lw=1, ls=':' )
ax.plot( [ax3, ax3], [ay3, -f], color=COLORS[1], lw=1, ls='-' )
#ax.text( ax3, -1.05*f, "$r_\mathrm{BFP}$", va='top', ha='center', color=COLORS[2] )
#ax.text( ax3, -1.05*f, "$r$", va='top', ha='center', color=COLORS[3] )
MEAS_r = Curve().scale(ax2,0).translate(0, ay2).set_attrib(color=COLORS[1], lw=1, ls='-')
MEAS_r.draw(ax)
MEAS_r.label(ax, "$r$", 'bottom', m=0.1)
#ax.plot( [ax2, 0], [ay2, ay2],  )
#ax.text( ax3, -1.05*f, "$r_\mathrm{BFP}$", va='top', ha='center', color=COLORS[2] )
#ax.text( ax2/2, ay2-0.05, "$r$", va='top', ha='center', color=COLORS[3] )

_THETA = np.linspace( 0, a, 100 )
ang_x = 7*ro*np.sin(_THETA)
ang_y = -7*ro*np.cos(_THETA)+f
ax.plot( ang_x, ang_y, color=COLORS[1], lw=1, ls='-' )
#ax.text( ang_x[-1], ang_y[-1], " $α$", va='bottom', ha='left', color=COLORS[2] )
ax.text( 0.5*( ang_x[-1] + ang_x[0] ), 
         0.5*( ang_y[-1] + ang_y[0] ), 
         "$θ$", va='bottom', ha='center', color=COLORS[1] )


path_between(OL, TL, r, r, 0).draw(ax, c=COLORS[2], ls=':')
path_between(OL, TL, r, r, 1).draw(ax, c=COLORS[2], ls=':')
path_between(TL, EMCCD1, r, 0, 0).draw(ax, c=COLORS[2], ls=':')
path_between(TL, EMCCD1, r, 0, 1).draw(ax, c=COLORS[2], ls=':')

path_between(BFP, M1, 0, 0.707*r/2, 0).draw(ax, c=COLORS[0], ls='-')
path_between(BFP, M1, 0, 0.707*r/2, 1).draw(ax, c=COLORS[0], ls='-')
path_between(M1, IL1, 0.707*r/2, r/2, 0).draw(ax, c=COLORS[0], ls='-')
path_between(M1, IL1, 0.707*r/2, r/2, 1).draw(ax, c=COLORS[0], ls='-')
path_between(IL1, IL2, r/2, r/2, 0).draw(ax, c=COLORS[0], ls='-')
path_between(IL1, IL2, r/2, r/2, 1).draw(ax, c=COLORS[0], ls='-')
path_between(IL2, M2, r/2, -0.75*r/2, 0).draw(ax, c=COLORS[0], ls='-')
path_between(IL2, M2, r/2, -0.75*r/2, 1).draw(ax, c=COLORS[0], ls='-')

path_between(IL2, M2, 0, 0, 0).draw(ax, c=COLORS[1], ls='--')
path_between(M2, SLIT, 0, 0, 0).draw(ax, c=COLORS[1], ls='--')
path_between(M2, SLIT, 0, 0, 0).label(ax, "Path length must\nremain fixed.", c=COLORS[1])

path_between(M2, OL3, -0.75*r/2, -0.3*r/2, 0).draw(ax, c=COLORS[0], ls='-')
path_between(M2, OL3, -0.75*r/2, -0.3*r/2, 1).draw(ax, c=COLORS[0], ls='-')
path_between(OL3, TL3, 0.3*r/2, 0.3*r/2, 0).draw(ax, c=COLORS[0], ls='-')
path_between(OL3, TL3, 0.3*r/2, 0.3*r/2, 1).draw(ax, c=COLORS[0], ls='-')
path_between(TL3, EMCCD2, 0.3*r/2, 0, 0).draw(ax, c=COLORS[0], ls='-')
path_between(TL3, EMCCD2, 0.3*r/2, 0, 1).draw(ax, c=COLORS[0], ls='-')
EMCCD2.translate(-0.5*f, 0)
path_between(GRATING, EMCCD2, 0.1*r/2, 0, 0).draw(ax, c=COLORS[0], ls=':')
path_between(GRATING, EMCCD2, 0.1*r/2, 0, 1).draw(ax, c=COLORS[0], ls=':')

#path_between(SLIT, OL3, 0, 0.5*r, 0).draw(ax, c=COLORS[0], ls=':')
#path_between(SLIT, OL3, 0, 0.5*r, 1).draw(ax, c=COLORS[0], ls=':')


#ax.grid()
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig.savefig("sketches/{style}/back-focal-plane-imaging-in-a-nutshell.pdf".format(style=PLOTSTYLE), bbox_inches='tight', dpi=100)